In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import xgboost as xgb
import numpy as np
import pandas as pd
import tensorflow as tf
"""
'inc_ability': 考虑到您的能力和工作状况，您目前的收入是否合理
'view': 根据您的一般印象您对一些重要事情所持的观点和看法与社会大众一致的时候有多少 
'status_3_before': 与三年前相比，您的社会经济地位发生了什么变化
'status_peer': 与同龄人相比，您本人的社会经济地位怎样
"""

In [2]:
data_dir = '/Users/changqingli/pycharm_projects/algorithms/weekend_content/'
train_data_file = 'data/happiness_train_abbr.csv'
test_data_dile = 'data/happiness_test_abbr.csv'

In [3]:
df_train = pd.read_csv(data_dir+train_data_file)
df_test = pd.read_csv(data_dir+test_data_dile)
df_train.head()

,id,happiness,survey_type,province,city,county,survey_time,gender,birth,nationality,...,family_income,family_m,family_status,house,car,marital,status_peer,status_3_before,view,inc_ability
0,1,4,1,12,32,59,2015/8/4 14:18,1,1959,1,...,60000.0,2,2,1,2,3,3,2,4,3
1,2,4,2,18,52,85,2015/7/21 15:04,1,1992,1,...,40000.0,3,4,1,2,1,1,1,4,2
2,3,4,2,29,83,126,2015/7/21 13:24,2,1967,1,...,8000.0,3,3,1,2,3,2,1,4,2
3,4,5,2,10,28,51,2015/7/25 17:33,2,1943,1,...,12000.0,3,3,1,1,7,2,1,3,2
4,5,4,1,7,18,36,2015/8/10 9:50,2,1994,1,...,-2.0,4,3,1,1,1,3,2,3,-8


In [4]:


df_train = df_train.drop('survey_time', 1).drop('id', 1)
df_train = df_train[df_train['happiness'] != -8]
df_train = df_train.fillna(0)

df_test = df_test.drop('survey_time', 1)
df_test = df_test.fillna(0)



In [5]:
df_train_train = df_train.sample(frac=0.8, random_state=0, axis=0)
df_train_validation = df_train[~df_train.index.isin(df_train_train.index)]



In [6]:
tr_data = df_train_train.as_matrix()
va_data = df_train_validation.as_matrix()
te_data = df_test.as_matrix()
tr_data


/Users/changqingli/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.
/Users/changqingli/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
/Users/changqingli/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until


array([[ 3.,  1.,  1., ...,  2.,  3., -8.],
       [ 5.,  2., 22., ...,  1.,  4.,  2.],
       [ 5.,  1., 21., ...,  1.,  4.,  2.],
       ...,
       [ 4.,  1., 17., ...,  2.,  3.,  2.],
       [ 3.,  2., 29., ...,  2.,  4.,  3.],
       [ 5.,  1.,  1., ...,  2.,  4.,  2.]])

In [7]:
tr_x_data = tr_data[:, 1:]
tr_y_data = tr_data[:, 0]

va_x_data = va_data[:, 1:]
va_y_data = va_data[:, 0]

In [8]:
len(va_x_data)

1598

In [9]:
from sklearn import linear_model
mod = linear_model.LinearRegression()

In [10]:
mod.fit(tr_x_data, tr_y_data)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [11]:
va_y_pred = np.round(mod.predict(va_x_data))

In [12]:
print(va_y_data)
print(va_y_pred)
print(va_y_pred - va_y_data)
print(np.mean((va_y_pred - va_y_data)*(va_y_pred - va_y_data)))

[4. 4. 5. ... 3. 5. 4.]
[4. 4. 3. ... 4. 4. 4.]
[ 0.  0. -2. ...  1. -1.  0.]
0.5901126408010012


In [13]:
pred_data = te_data[:, 1:]
pred_data

array([[ 1.,  2.,  2., ...,  1.,  4.,  2.],
       [ 1., 22., 66., ...,  3.,  4.,  2.],
       [ 2.,  9., 22., ...,  1.,  3.,  4.],
       ...,
       [ 2., 29., 84., ...,  1.,  4.,  2.],
       [ 1., 11., 31., ...,  1.,  4., -8.],
       [ 1., 13., 37., ...,  2.,  4.,  2.]])

In [14]:
pred_res = np.round(mod.predict(pred_data))

In [15]:
pred_res

array([4., 3., 4., ..., 4., 4., 5.])

In [16]:
import csv
result = 'submission_908.csv'
with open(data_dir+result, 'w') as f:
    writer = csv.writer(f, delimiter=',')
    writer.writerow(['id', 'happiness'])
    for i in range(len(te_data)):
        writer.writerow([int(te_data[i][0]), int(pred_res[i])])

In [32]:
len(mod.coef_)

39

In [38]:
col_name = list(df_train_train.columns)[1:]
m = {}
for i in range(len(col_name)):
    m.setdefault(col_name[i], 0)
    m[col_name[i]] = mod.coef_[i]
    
sorted(m.items(), key=lambda x: x[1], reverse=True)
    

[('inc_ability', 0.1478203254716955),
 ('view', 0.1354216854120923),
 ('status_3_before', 0.11309520348465896),
 ('status_peer', 0.10897358939256352),
 ('marital', 0.05896973794058932),
 ('car', 0.05764759360934971),
 ('house', 0.04985383730834656),
 ('family_status', 0.047524919959526346),
 ('family_m', 0.045190326914817544),
 ('family_income', 0.03887617524370268),
 ('work_manage', 0.02828830546184053),
 ('work_type', 0.014266987114460963),
 ('work_yr', 0.012533116360491458),
 ('work_status', 0.011805566314329358),
 ('work_exper', 0.01089302384506575),
 ('class', 0.009085872945616974),
 ('equity', 0.008918152252317343),
 ('learn', 0.005594841096430921),
 ('relax', 0.004828914055028089),
 ('socialize', 0.004344556716593074),
 ('hukou', 0.0038397647481808656),
 ('depression', 0.0029490341002829682),
 ('health_problem', 2.276171106051765e-05),
 ('health', 1.655793586960237e-08),
 ('weight_jin', 5.313719810329362e-09),
 ('height_cm', -7.64840580379878e-05),
 ('floor_area', -0.00128828891